In [1]:
from typing import NamedTuple, Optional

class Data(NamedTuple):
  name: str
  size: Optional[float]
  state: Optional[bool]
  date: Optional[str]
  time: Optional[str]

In [2]:
def generate_data():
  yield Data("3D Objects", 35.1, True, "06/06/2020", "00:55")
  yield Data(".pylint.d", 28.85, False, "03/17/2021", "23:46")
  yield Data("Favorites", None, None, None, None)
  yield Data("temp", 19.64, True, "05/14/2020", "22:39")
  yield Data(".gitconfig", 0.74, True, "02/28/2021", "22:55")
  yield Data(".bashrc", 4.12, True, "02/28/2021", "21:55")

## 格式化要求

1. 直接输出name
2. size保留一位小数，后面加MB，默认值0.0MB
3. 根据size，输出字符串，默认值Small
  - `<10MB: Small`
  - `>=10MB, <20MB: Medium`
  - `>=20MB: Big`
4. 根据state，输出字符串，默认值Success
  - True：Success
  - False：Failure
5. 拼接date和time，中间加空格，默认值Unknown


In [3]:
def format_name(data):
  return data.name


def format_abs_size(data):
  if data.size is None:
    return '0.0MB'

  return '{:.1f}MB'.format(data.size)


def format_relative_size(data):
  if data.size is None:
    return 'Small'

  if data.size < 10:
    return 'Small'
  elif data.size < 20:
    return 'Medium'
  else:
    return 'Big'


def format_state(data):
  if data.state is None:
    return 'Success'

  return 'Success' if data.state else 'Failure'


def format_date_time(data):
  if data.date is None or data.time is None:
    return 'Unknown'

  return data.date + ' ' + data.time

In [4]:
def generate_a_row(data):
  for f in (
    format_name,
    format_abs_size,
    format_relative_size,
    format_state,
    format_date_time
  ):
    yield f(data)


In [5]:
from pprint import pprint

pprint(list(
  map(
    tuple,
    map(generate_a_row, generate_data())
  )
))

[('3D Objects', '35.1MB', 'Big', 'Success', '06/06/2020 00:55'),
 ('.pylint.d', '28.9MB', 'Big', 'Failure', '03/17/2021 23:46'),
 ('Favorites', '0.0MB', 'Small', 'Success', 'Unknown'),
 ('temp', '19.6MB', 'Medium', 'Success', '05/14/2020 22:39'),
 ('.gitconfig', '0.7MB', 'Small', 'Success', '02/28/2021 22:55'),
 ('.bashrc', '4.1MB', 'Small', 'Success', '02/28/2021 21:55')]


## 函数的状态（一）

如果要求函数各个参数（单位、字符串、分隔符等）可以定义，函数签名会变成：

```python
def format_abs_size(data, unit, default):
  ...

def format_relative_size(data, strings, default):
  ...

def format_date_time(data, sep, default):
  ...
```

将导致一些不便：

- 函数入参个数、含义不定，最终调用不方便
- 函数参数分为数据（data）和不随数据变化的值（unit、strings、sep、default等），传参时机不同。

如果将不随数据变化的值作为状态保存，就能够消除这种不便。先以我们熟悉的OO模式为例。

In [9]:
class DateTimeFormatter:
  def __init__(self, sep=' ', default='Unknown'):
    self.sep = sep
    self.default = default
  
  def format(self, data):
    if data.date is None or data.time is None:
      return self.default

    return data.date + self.sep + data.time


formatter = DateTimeFormatter('/', 'N/A')
for data in generate_data():
  print(formatter.format(data))

06/06/2020/00:55
03/17/2021/23:46
N/A
05/14/2020/22:39
02/28/2021/22:55
02/28/2021/21:55


在Python中，可以结合OO和FP，把类实例“模拟”成函数。方便和其他FP技术、风格并存，或者复用其他FP技巧、工具。

In [10]:
class DateTimeFormatter:
  def __init__(self, sep=' ', default='Unknown'):
    self.sep = sep
    self.default = default
  
  def __call__(self, data):
    if data.date is None or data.time is None:
      return self.default

    return data.date + self.sep + data.time


formatter = DateTimeFormatter('/', 'N/A')
for data in generate_data():
  print(formatter(data))

06/06/2020/00:55
03/17/2021/23:46
N/A
05/14/2020/22:39
02/28/2021/22:55
02/28/2021/21:55


In [11]:
def generate_a_row_new_style(data):
  date_time_formatter = DateTimeFormatter('/', 'N/A')
  for f in (
    format_name,
    format_abs_size,
    format_relative_size,
    format_state,
    date_time_formatter
  ):
    yield f(data)


from pprint import pprint

pprint(list(
  map(
    tuple,
    map(generate_a_row_new_style, generate_data())
  )
))

[('3D Objects', '35.1MB', 'Big', 'Success', '06/06/2020/00:55'),
 ('.pylint.d', '28.9MB', 'Big', 'Failure', '03/17/2021/23:46'),
 ('Favorites', '0.0MB', 'Small', 'Success', 'N/A'),
 ('temp', '19.6MB', 'Medium', 'Success', '05/14/2020/22:39'),
 ('.gitconfig', '0.7MB', 'Small', 'Success', '02/28/2021/22:55'),
 ('.bashrc', '4.1MB', 'Small', 'Success', '02/28/2021/21:55')]
